In [ ]:
#Project Template

In [ ]:
#Dependancies
import pandas as pd
import datetime
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import scipy.stats as stats

In [ ]:
#Data Cleaning (Joe and Anthony)

# Joe

In [ ]:
#Cleaning the pollution csv

In [ ]:
file = "dataset/cleanpollution2011-2016.csv"

In [ ]:
#look at the file
df = pd.read_csv(file)
df.head()

In [ ]:
#Filter the date range I want (2014-2016)
date_df = df.loc[(df['Date Local'] > '2013-12-31')]

date_df.head()

In [ ]:
#Pulling out the states I want
statecities_df = date_df
state_list =['California', 'Texas', 'Florida','New Jersey','Illinois', 'Colorado', 'North Carolina']


In [ ]:
#Filtered the States
statecities_df[statecities_df.State.isin(state_list)]

In [ ]:
#Filtering the Cities
city_list = ['Los Angeles', 'Dallas', 'Winter Park', 'Newark', 'Chicago', 'Denver', 'Charlotte']

In [ ]:
statecities_df[statecities_df.City.isin(city_list)]

In [ ]:
#remove duplicate readings as the data summary nevere clears up why there is multiple readings

In [ ]:
cleanstatecities_df = statecities_df
# .drop_duplicates(subset ='Date Local', keep = 'first')

In [ ]:
cleanstatecities_df

In [ ]:
cleanstatecities_df.to_csv ("dataset/cleanpollution14-2016.csv", index=False, header=True)


In [ ]:
# upload cleanpollution14-2016.csv
file = "dataset/cleanpollution14-2016.csv"
cleanpollution = pd.read_csv(file)
cleanpollution.count()


In [ ]:
# Convert Date Local to datetime
cleanpollution['Date Local'] = pd.to_datetime(cleanpollution['Date Local'], infer_datetime_format=True)
cleanpollution.head()

In [ ]:
# Averaging pollution data on weekly basis
cleanpollution = cleanpollution.groupby(['State', pd.Grouper(key='Date Local', freq='W-MON')]) \
       .mean() \
       .reset_index() \
       .sort_values('Date Local')
cleanpollution

In [ ]:
#Clean the gas price csv
gasfile = "dataset/gasprices.csv"

In [ ]:
gasdata = pd.read_csv(gasfile)
gasdata

In [ ]:
#Grabbing the rows I want
gasdata.columns

In [ ]:
gasdata_reduced = gasdata.loc[:,['Date', 'A2']]
gasdata_reduced.head()

In [ ]:
gasdata_reduced = gasdata_reduced.rename(columns={"A2":"Gas Prices"}) 
gasdata_reduced.head()

In [ ]:
gasdata_reduced.count()

In [ ]:
# convert Date to datetime
gasdata_reduced['Date'] = pd.to_datetime(gasdata_reduced['Date'])
print(gasdata_reduced)

In [ ]:
# Rename Date to Date Local to create a similar key variable for merging pollution and gas files
gasdata_reduced.rename(columns={'Date': 'Date Local'}, inplace=True)
print(gasdata_reduced)

In [ ]:
# merge weekly pollution and gas data
combinedfiles = pd.merge(left=cleanpollution, right=gasdata_reduced, left_on='Date Local', right_on='Date Local')
combinedfiles

In [ ]:
#Regression (Simmy/Nate)

In [ ]:
#Get Data For Each State 'Califorina', 'Texas', 'Florida','New Jersey','Illinois', 'Colorado', 'North Carolina'
StateSubset=combinedfiles[combinedfiles["State"].isin(state_list)]

In [ ]:
#We only want the date with all states represented
Datesorted=StateSubset.groupby(["Date Local"]).count()
Datesorted=Datesorted[Datesorted["State"]==7]
testabledates=Datesorted.index
testabledates
#testabledates are all the dates which have 7 entries (which are all the states)

In [ ]:
#Now sum the results for all states and group by the dates. Only include dates in testabledates
TotalStates=StateSubset[StateSubset["Date Local"].isin(testabledates)]
TotalStates=TotalStates.groupby("Date Local").sum()
TotalStates["Gas Prices"]=TotalStates["Gas Prices"]/7
#We divide "Gas Prices" by 7 since we summed it 7 times.
TotalStates

In [ ]:
#We can also look at Individual States
CombinedStates=StateSubset.groupby(["State", "Date Local"]).sum()
CombinedStates

In [ ]:
#NO2 For every state
for state in state_list:
    statedata=CombinedStates.loc[state]
    x_values = statedata['Gas Prices']
    y_values = statedata['NO2 Mean']
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,(2,max(y_values)*0.9),fontsize=15,color="red")
    plt.xlabel('Gas Price ($)')
    plt.ylabel('NO2 Emission')
    plt.title(f"NO2 Emission by Gas Price for {state}")
    plt.savefig(f'Regression Output/States/{state}/NO2{state}.png')
    plt.show()

In [ ]:
#SO2 For every state
for state in state_list:
    statedata=CombinedStates.loc[state]
    x_values = statedata['Gas Prices']
    y_values = statedata['SO2 Mean']
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,(2,max(y_values)*0.9),fontsize=15,color="red")
    plt.xlabel('Gas Price ($)')
    plt.ylabel('SO2 Emission')
    plt.title(f"SO2 Emission by Gas Price for {state}")
    plt.savefig(f'Regression Output/States/{state}/SO2{state}.png')
    plt.show()

In [ ]:
#O3 For every state
for state in state_list:
    statedata=CombinedStates.loc[state]
    x_values = statedata['Gas Prices']
    y_values = statedata['O3 Mean']*100
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,(2,max(y_values)*0.9),fontsize=15,color="red")
    plt.xlabel('Gas Price ($)')
    plt.ylabel('O3 Emission (X100)')
    plt.title(f"O3 Emission by Gas Price for {state}")
    plt.savefig(f'Regression Output/States/{state}/O3{state}.png')
    plt.show()

In [ ]:
#CO For every state
for state in state_list:
    statedata=CombinedStates.loc[state]
    x_values = statedata['Gas Prices']
    y_values = statedata['CO Mean']*10
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,(2,max(y_values)*0.9),fontsize=15,color="red")
    plt.xlabel('Gas Price ($)')
    plt.ylabel('CO Emission (X10)')
    plt.title(f"CO Emission by Gas Price for {state}")
    plt.savefig(f'Regression Output/States/{state}/CO{state}.png')
    plt.show()

In [ ]:
x_values = TotalStates['Gas Prices']
y_values = TotalStates['NO2 Mean']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(2,40),fontsize=15,color="red")
plt.xlabel('Gas Price ($)')
plt.ylabel('NO2 Emission')
plt.title("NO2 Emission by Gas Price for Combined States")
plt.savefig('Regression Output/NO2Total.png')
plt.show()

In [ ]:
x_values = TotalStates['Gas Prices']
y_values = TotalStates['SO2 Mean']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(2,2),fontsize=15,color="red")
plt.xlabel('Gas Price ($)')
plt.ylabel('SO2 Emission')
plt.title("SO2 Emission by Gas Price for Combined States")
plt.savefig('Regression Output/SO2Total.png')
plt.show()

In [ ]:
x_values = TotalStates['Gas Prices']
y_values = TotalStates['O3 Mean']*100
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(2,25),fontsize=15,color="red")
plt.xlabel('Gas Price ($)')
plt.ylabel('O3 Emission (X100)')
plt.title("O3 Emission by Gas Price for Combined States")
plt.savefig('Regression Output/O3Total.png')
plt.show()

In [ ]:
x_values = TotalStates['Gas Prices']
y_values = TotalStates['CO Mean']*10
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(2,15),fontsize=15,color="red")
plt.xlabel('Gas Price ($)')
plt.ylabel('CO Emission (X10)')
plt.title("CO Emission by Gas Price for Combined States")
plt.savefig('Regression Output/COTotal.png')
plt.show()

In [ ]:
#Hypothesis Testing (Nate)

In [ ]:
#Sort TotalStates by Gas Prices, group by High, Middle, and Low
sortedTotalStates=TotalStates.sort_values(by=["Gas Prices"])
dflength=len(sortedTotalStates["Gas Prices"])
low=sortedTotalStates.iloc[0:int(dflength/3),:]
middle=sortedTotalStates.iloc[int(dflength/3):int(2*dflength/3),:]
high=sortedTotalStates.iloc[int(2*dflength/3):,:]

In [ ]:
p_value=stats.ttest_ind(low["NO2 Mean"], high["NO2 Mean"], equal_var=False)[1]
print(f"The p value for NO2 is {p_value}")
p_value=stats.ttest_ind(low["O3 Mean"], high["O3 Mean"], equal_var=False)[1]
print(f"The p value for O3 is {p_value}")
p_value=stats.ttest_ind(low["SO2 Mean"], high["SO2 Mean"], equal_var=False)[1]
print(f"The p value for SO2 is {p_value}")
p_value=stats.ttest_ind(low["CO Mean"], high["CO Mean"], equal_var=False)[1]
print(f"The p value for CO is {p_value}")

In [ ]:
#charts (simmy)


In [ ]:
CaliforniaNO2 = CombinedStates.loc["California"]["NO2 Mean"].sum()
COLORADONO2 = CombinedStates.loc["Colorado"]["NO2 Mean"].sum()
TexasNO2 = CombinedStates.loc["Texas"]["NO2 Mean"].sum()
NewJerseyNO2= CombinedStates.loc["New Jersey"]["NO2 Mean"].sum()
IllinoisNO2= CombinedStates.loc["Illinois"]["NO2 Mean"].sum()
FloridaNO2= CombinedStates.loc["Florida"]["NO2 Mean"].sum()
NorthCarolinaNO2= CombinedStates.loc["North Carolina"]["NO2 Mean"].sum()
NorthCarolinaNO2

In [ ]:
# Labels for the sections of our pie chart 
labels = ['California', 'Texas', 'Florida','New Jersey','Illinois', 'Colorado', 'North Carolina']

# The values of each section of the pie chart
sizes = [CaliforniaNO2,TexasNO2,FloridaNO2,NewJerseyNO2,IllinoisNO2,COLORADONO2,NorthCarolinaNO2]

#The colors of each section of the pie chart
colors = ["red", "orange", "gray", "green","yellow","lavender","lightskyblue"]

#Tells matplotlib to seperate the section from the others 
explode = (0,0,0,0,0,0.1,0)

In [ ]:
# Creates the pie chart based upon the values above
# Automatically finds the percentages of each part of the pie chart
plt.pie(sizes, explode=explode, labels=labels, colors=colors,        
        autopct="%1.1f%%", shadow=True, startangle=140) 
plt.title("NO2 Mean by State")

In [ ]:
CaliforniaO3 = CombinedStates.loc["California"]["O3 Mean"].sum()
COLORADOO3 = CombinedStates.loc["Colorado"]["O3 Mean"].sum()
TexasO3 = CombinedStates.loc["Texas"]["O3 Mean"].sum()
NewJerseyO3= CombinedStates.loc["New Jersey"]["O3 Mean"].sum()
IllinoisO3= CombinedStates.loc["Illinois"]["O3 Mean"].sum()
FloridaO3= CombinedStates.loc["Florida"]["O3 Mean"].sum()
NorthCarolinaO3= CombinedStates.loc["North Carolina"]["O3 Mean"].sum()

# Labels for the sections of our pie chart 
labels = ['California', 'Texas', 'Florida','New Jersey','Illinois', 'Colorado', 'North Carolina']

# The values of each section of the pie chart
sizes = [CaliforniaO3,TexasO3,FloridaO3,NewJerseyO3,IllinoisO3,COLORADOO3,NorthCarolinaO3]

#The colors of each section of the pie chart
colors = ["red", "orange", "gray", "green","yellow","lavender","lightskyblue"]

#Tells matplotlib to seperate the section from the others 
explode = (0,0,0,0,0,0,0.1)

# Creates the pie chart based upon the values above
# Automatically finds the percentages of each part of the pie chart
plt.pie(sizes, explode=explode, labels=labels, colors=colors,        
        autopct="%1.1f%%", shadow=True, startangle=140) 
plt.title("O3 Mean by State")



In [ ]:
CaliforniaSO2 = CombinedStates.loc["California"]["SO2 Mean"].sum()
COLORADOSO2 = CombinedStates.loc["Colorado"]["SO2 Mean"].sum()
TexasSO2 = CombinedStates.loc["Texas"]["SO2 Mean"].sum()
NewJerseySO2= CombinedStates.loc["New Jersey"]["SO2 Mean"].sum()
IllinoisSO2= CombinedStates.loc["Illinois"]["SO2 Mean"].sum()
FloridaSO2= CombinedStates.loc["Florida"]["SO2 Mean"].sum()
NorthCarolinaSO2= CombinedStates.loc["North Carolina"]["SO2 Mean"].sum()

# Labels for the sections of our pie chart 
labels = ['California', 'Texas', 'Florida','New Jersey','Illinois', 'Colorado', 'North Carolina']

# The values of each section of the pie chart
sizes = [CaliforniaSO2,TexasSO2,FloridaSO2,NewJerseySO2,IllinoisSO2,COLORADOSO2,NorthCarolinaSO2]

#The colors of each section of the pie chart
colors = ["red", "orange", "gray", "green","yellow","lavender","lightskyblue"]

#Tells matplotlib to seperate the section from the others 
explode = (0,0,0,0,0,0.1,0)

# Creates the pie chart based upon the values above
# Automatically finds the percentages of each part of the pie chart
plt.pie(sizes, explode=explode, labels=labels, colors=colors,        
        autopct="%1.1f%%", shadow=True, startangle=140) 
plt.title("SO2 Mean by State")

In [ ]:
CaliforniaCO = CombinedStates.loc["California"]["CO Mean"].sum()
COLORADOCO = CombinedStates.loc["Colorado"]["CO Mean"].sum()
TexasCO = CombinedStates.loc["Texas"]["CO Mean"].sum()
NewJerseyCO= CombinedStates.loc["New Jersey"]["CO Mean"].sum()
IllinoisCO= CombinedStates.loc["Illinois"]["CO Mean"].sum()
FloridaCO= CombinedStates.loc["Florida"]["CO Mean"].sum()
NorthCarolinaCO= CombinedStates.loc["North Carolina"]["CO Mean"].sum()

# Labels for the sections of our pie chart 
labels = ['California', 'Texas', 'Florida','New Jersey','Illinois', 'Colorado', 'North Carolina']

# The values of each section of the pie chart
sizes = [CaliforniaCO,TexasCO,FloridaCO,NewJerseyCO,IllinoisCO,COLORADOCO,NorthCarolinaCO]

#The colors of each section of the pie chart
colors = ["red", "orange", "gray", "green","yellow","lavender","lightskyblue"]

#Tells matplotlib to seperate the section from the others 
explode = (0,0,0.1,0,0,0,0)

# Creates the pie chart based upon the values above
# Automatically finds the percentages of each part of the pie chart
plt.pie(sizes, explode=explode, labels=labels, colors=colors,        
        autopct="%1.1f%%", shadow=True, startangle=140) 
plt.title("CO Mean by State")

In [ ]:
 # The maximum x value for our chart will be 100
x_limit = 100
TotalStates 
# List of values from 0 to 100 each value being 1 greater than the last
x_axis = TotalStates.index
y_axis = TotalStates["NO2 Mean"]
TotalStates

# Create a random array of data that we will use for our y values
#data = [random.random() for value in x_axis]

In [ ]:
# Tells matplotlib that we want to make a scatter plot
# The size of each point on our plot is determined by their x value
plt.scatter(x_axis, y_axis, marker="o", facecolors="red", edgecolors="black",
             alpha=0.75)
plt.title("NO2 Mean by Date")
plt.ylabel("Emission parts by Billion")
plt.xlabel("Date")
plt.xticks(rotation=45)

In [ ]:
TotalStates.index

In [ ]:
 # The maximum x value for our chart will be 100
x_limit = 100
TotalStates 
# List of values from 0 to 100 each value being 1 greater than the last
x_axis = TotalStates.index
y_axis = TotalStates["O3 Mean"]
TotalStates

# Create a random array of data that we will use for our y values
#data = [random.random() for value in x_axis]

# Tells matplotlib that we want to make a scatter plot
# The size of each point on our plot is determined by their x value
plt.scatter(x_axis, y_axis, marker="o", facecolors="red", edgecolors="black",
             alpha=0.75)
plt.title("O3 Mean by Date")
plt.ylabel("Emission parts by Million")
plt.xlabel("Date")
plt.xticks(rotation=45)


In [ ]:
 # The maximum x value for our chart will be 100
x_limit = 100
TotalStates 
# List of values from 0 to 100 each value being 1 greater than the last
x_axis = TotalStates.index
y_axis = TotalStates["SO2 Mean"]
TotalStates

# Create a random array of data that we will use for our y values
#data = [random.random() for value in x_axis]

# Tells matplotlib that we want to make a scatter plot
# The size of each point on our plot is determined by their x value
plt.scatter(x_axis, y_axis, marker="o", facecolors="red", edgecolors="black",
             alpha=0.75)
plt.title("SO2 Mean by Date")
plt.ylabel("Emission parts by Billion")
plt.xlabel("Date")
plt.xticks(rotation=45)

In [ ]:
 # The maximum x value for our chart will be 100
x_limit = 100
TotalStates 
# List of values from 0 to 100 each value being 1 greater than the last
x_axis = TotalStates.index
y_axis = TotalStates["CO Mean"]

# Create a random array of data that we will use for our y values
#data = [random.random() for value in x_axis]

# Tells matplotlib that we want to make a scatter plot
# The size of each point on our plot is determined by their x value
plt.scatter(x_axis, y_axis, marker="o", facecolors="red", edgecolors="black",
             alpha=0.75)
plt.title("CO Mean by Date")
plt.ylabel("Emission parts by Million")
plt.xlabel("Date")
plt.xticks(rotation=45)